In [5]:
import requests
import json

url = 'http://localhost:8080/ctakes-web-rest/service/analyze'
params = {'pipeline': 'Default'}
headers = {'cache-control': 'no-cache'}
data = """
The patient is a 67-year-old male presenting to the emergency department with shortness of breath and a persistent cough for the past 48 hours. He reports a subjective fever and general malaise. The patient began experiencing exertional dyspnea two days ago, which has progressed to dyspnea at rest. He has a productive cough with yellow sputum. He denies chest pain or palpitations. He notes that he has had a runny nose for about a week. The patient's history is significant for hypertension diagnosed in 2010, managed with lisinopril 20mg daily. He also has a history of Type 2 Diabetes Mellitus, diagnosed in 2015, for which he takes metformin 1000mg twice a day. No known drug allergies. The patient's vitals are: BP 145/88, HR 95, Temp 100.2°F, RR 22, SpO2 90% on room air. Lungs show diminished breath sounds at the right lower lobe with crackles on auscultation. Cardiovascular is regular rate and rhythm, no murmurs. There is no peripheral edema. The patient's presentation is concerning for community-acquired pneumonia. We will initiate treatment with azithromycin 500mg IV stat and follow with 250mg PO daily. We will also order a chest X-ray and obtain a sputum culture to confirm the diagnosis and identify the causative organism. The patient will be admitted to the hospital for observation and management of his respiratory distress. We will continue his home medications as directed."""

try:
    response = requests.post(url, params=params, headers=headers, data=data)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

    print("Status Code:", response.status_code)
    print("Response Body:")
    print(response.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Status Code: 200
Response Body:
{"AnatomicalSiteMention":[{"begin":357,"end":362,"text":"chest","polarity":1,"conceptAttributes":[{"code":"51185008","cui":"C0817096","codingScheme":"SNOMEDCT_US","tui":"T029"},{"code":"261179002","cui":"C0817096","codingScheme":"SNOMEDCT_US","tui":"T029"}]},{"begin":418,"end":422,"text":"nose","polarity":1,"conceptAttributes":[{"code":"45206002","cui":"C0028429","codingScheme":"SNOMEDCT_US","tui":"T023"},{"code":"260540009","cui":"C0028429","codingScheme":"SNOMEDCT_US","tui":"T023"}]},{"begin":782,"end":787,"text":"Lungs","polarity":1,"conceptAttributes":[{"code":"39607008","cui":"C0024109","codingScheme":"SNOMEDCT_US","tui":"T023"}]},{"begin":825,"end":841,"text":"right lower lobe","polarity":1,"conceptAttributes":[{"code":"266005","cui":"C1261075","codingScheme":"SNOMEDCT_US","tui":"T023"}]},{"begin":831,"end":841,"text":"lower lobe","polarity":1,"conceptAttributes":[{"code":"90572001","cui":"C0225758","codingScheme":"SNOMEDCT_US","tui":"T023"}]},{"be

In [6]:
def pretty_print_ctakes_output(json_output):
    """
    Parses and prints the cTAKES JSON output in a human-readable format.

    Args:
        json_output (dict): The dictionary containing the cTAKES JSON data.
    """
    # Define a user-friendly mapping for the keys
    category_map = {
        "AnatomicalSiteMention": "Anatomical Sites 📍",
        "MedicationMention": "Medications 💊",
        "DiseaseDisorderMention": "Diseases & Disorders 🤒",
        "SignSymptomMention": "Signs & Symptoms 🤕",
        "ProcedureMention": "Procedures 🩺",
        "MeasurementAnnotation": "Measurements 📏",
        # You can add more categories here as needed
    }

    # Iterate through the main categories of the JSON output
    for key, human_name in category_map.items():
        if key in json_output and json_output[key]:
            print(f"--- {human_name} ---")
            
            # Iterate through each mention in the category
            for mention in json_output[key]:
                text = mention.get("text", "N/A")
                polarity = "Positive" if mention.get("polarity", 1) == 1 else "Negative"
                concept_attributes = mention.get("conceptAttributes", [])
                
                codes_and_schemes = []
                for attr in concept_attributes:
                    code = attr.get("code")
                    scheme = attr.get("codingScheme")
                    cui = attr.get("cui")
                    codes_and_schemes.append(f"({scheme}: {code}, CUI: {cui})")
                
                print(f"  - **{text}**")
                print(f"    - Polarity: {polarity}")
                if codes_and_schemes:
                    print(f"    - Codes: {', '.join(codes_and_schemes)}")
            print("\n")

In [7]:
pretty_print_ctakes_output(json.loads(response.text))

--- Anatomical Sites 📍 ---
  - **chest**
    - Polarity: Positive
    - Codes: (SNOMEDCT_US: 51185008, CUI: C0817096), (SNOMEDCT_US: 261179002, CUI: C0817096)
  - **nose**
    - Polarity: Positive
    - Codes: (SNOMEDCT_US: 45206002, CUI: C0028429), (SNOMEDCT_US: 260540009, CUI: C0028429)
  - **Lungs**
    - Polarity: Positive
    - Codes: (SNOMEDCT_US: 39607008, CUI: C0024109)
  - **right lower lobe**
    - Polarity: Positive
    - Codes: (SNOMEDCT_US: 266005, CUI: C1261075)
  - **lower lobe**
    - Polarity: Positive
    - Codes: (SNOMEDCT_US: 90572001, CUI: C0225758)
  - **Cardiovascular**
    - Polarity: Positive
    - Codes: (SNOMEDCT_US: 113257007, CUI: C0007226)
  - **chest**
    - Polarity: Positive
    - Codes: (SNOMEDCT_US: 51185008, CUI: C0817096), (SNOMEDCT_US: 261179002, CUI: C0817096)


--- Medications 💊 ---
  - **lisinopril**
    - Polarity: Positive
    - Codes: (SNOMEDCT_US: 386873009, CUI: C0065374), (RXNORM: 29046, CUI: C0065374), (RXNORM: 29046, CUI: C0065374), (SNO